# K-MEANS CLUSTERING WITH SOCIAL NETWORKING DATASET

Adapted from Lantz (2015) Chapter 9

For this analysis, we will use a dataset representing a random sample of 30,000 U.S. high school students who had profiles on a well-known SNS in 2006.

To protect the users' anonymity, the SNS will remain unnamed.

The data was sampled evenly across four high school graduation years (2006 through 2009) representing the senior, junior, sophomore, and freshman classes at the time of data collection. Using an automated web crawler, the full text of the SNS profiles were downloaded, and each teen's gender, age, and number of SNS friends was recorded.

A text mining tool was used to divide the remaining SNS page content into words. From the top 500 words appearing across all the pages, 36 words were chosen to represent five categories of interests: namely extracurricular activities, fashion, religion, romance, and antisocial behavior. The 36 words include terms such as football, sexy, kissed, bible, shopping, death, and drugs. The final dataset indicates, for each person, how many times each word appeared in the person's SNS profile.

## Load libraries and data

In [ ]:
library(data.table) # to handle the data in a more convenient manner
library(tidyverse) # for a better work flow and more tools to wrangle and visualize the data
library(plotly) # for interactive visualizations
library(fastDummies) # to create dummies from categoric variables
library(mice) # for missing values
library(VIM) # for visualizing missing values
library(BBmisc) # for standardization
library(formattable) # for number formatting
library(pheatmap) # heatmap
library(heatmaply) # heatmap
library(factoextra) # visualizing distances, cluster, heatmap
library(knitr) # pretty tables
library(kableExtra) # pretty tables
library(IRdisplay) # pretty tables
library(NbClust) # cluster metrics
library(vegan) # cluster metrics
library(listviewer) # view list object
library(plyr) # for lapply on matrix objects

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

In [ ]:
datapath <- "~/data_ad454"

Load the data:

In [ ]:
teens_dt <- data.table::fread(sprintf("%s/csv/11_01_snsdata.csv", datapath), stringsAsFactors = T)

## Explore data

View the structure:

In [ ]:
str(teens_dt)

The data include 30,000 teenagers with four variables indicating personal characteristics and 36 words indicating interests.

See factor levels:

In [ ]:
teens_dt %>% purrr::keep(is.factor) %>% purrr::map(levels)

And the distribution:

In [ ]:
teens_factors <- teens_dt %>% purrr::keep(is.factor) %>% # select factor columns
    tidyr::gather() %>% # convert into long format for faceting
    ggplot(aes(x = value)) + # plot value
    facet_wrap(~ key, scales = "free") + # divide into separate plots by key
    geom_bar()

plotly::ggplotly(teens_factors)

See numeric variables'distritubions (excluding NA's):

In [ ]:
teens_dt %>% purrr::keep(is.numeric) %>% sapply(quantile, na.rm = T) %>% t()

In [ ]:
teens_dt[,quantile(age, na.rm = T)]

### Cleaning age variable

Age variables has NA values:

In [ ]:
teens_dt %>%
    purrr::keep(is.numeric) %>%
    sapply(function(x) sum(is.na(x))) %>%
    "["(. > 0)

And age distribution is not logical for college students:

In [ ]:
teens_dt[,quantile(age, na.rm = T)]

We should leave only values 13-20:

In [ ]:
teens_dt[!dplyr::between(age, 13, 20 - 1e-10), age := NA]

Now more reasonable:

In [ ]:
teens_dt[,quantile(age, na.rm = T)]

### Dummify gender

In [ ]:
teens_dt[, gender := relevel(gender, "M")]

Gender variable has NA values:

In [ ]:
teens_dt[,table(gender,useNA = "ifany")]

Now let's create dummy variables for female and missing gender values:

In [ ]:
gender_dummy <- teens_dt[,fastDummies::dummy_cols(.(gender = gender),
                            remove_first_dummy = T)] %>% dplyr::select(-gender) %>%  mutate(gender_F = ifelse(is.na(gender_F), 0, gender_F))
gender_dummy

And insert these columns after the original gender variable:

In [ ]:
teens_dt <- teens_dt %>%
    append(unclass(gender_dummy), after = 2) %>% 
    as.data.table

In [ ]:
teens_dt

## Data imputation

Let's look at missing values and distribution of complete and incomplete cases:

In [ ]:
mdpat <- mice::md.pattern(teens_dt)
mdpat

In this output 1 means data exists and 0 means it is missing (NA). So:

- 23602 cases are complete
- In 3674 cases age is missing
- In 875 cases gender is missing
- In 1849 cases both age and gender are missing

Lets only filter for those columns which have missing values (using the last row in the output):

In [ ]:
mdcols <- mdpat %>%
            as.data.table %>%
            .[.N, names(.)[.SD > 0]] %>%
            head(-1)
mdcols

See whether we got it:

In [ ]:
teens_dt[,.SD, .SDcols = mdcols]

Now let's visualize the distribution of missing values in these columns:

First using tidyverse notation:

In [ ]:
teens_dt %>%
    dplyr::select(mdcols) %>%
    VIM::aggr(numbers = T)

And then in pure data.table:

In [ ]:
teens_dt[,VIM::aggr(.SD, numbers = T), .SDcols = mdcols]

### Imputation using mice package:

"pmm" method is predictive mean matching. It uses other columns' values distribution and respective mean values of the column with missing values in order to fill in those missing values:

mice() function gives the results separately

In this examples, missing age values are completed using gradyear:

In [ ]:
datatemp <- mice::mice(teens_dt[,.(gradyear, age)], method = "pmm", m = 1)

In [ ]:
summary(datatemp)

And we can see the imputed values for age:

In [ ]:
datatemp$imp$age

Impute missing values into completed object:

In [ ]:
completed <- mice::complete(datatemp,1)

Let's see, for the original missing values, with what values age was imputed with for each greadyear, first only averages:

In [ ]:
cbind(teens_dt[,.(age_original = age)], completed)[is.na(age_original), mean(age), by = gradyear]

And then the distribution of the imputed values for each gradyear:

In [ ]:
colnms <- quantile(1) %>% names
colnms

In [ ]:
cbind(teens_dt[,.(age_original = age)], completed)[
    is.na(age_original),
    .(colnms = colnms, age = quantile(age)),
    by = gradyear] %>%
tidyr::spread(colnms, age) %>%
select(c("gradyear", colnms))

#data.table::setcolorder(c("gradyear", colnms)) %>%
#.[]

We see that mice does not use fixed values for each gradyear adds some statistical distribution

### Imputation using custom function

We can also use fixed values for imputing while following common methods such as mean:

In [ ]:
cbind(teens_dt[,.(gradyear, age)], age2 = teens_dt[, ifelse(is.na(age), mean(age, na.rm = T), age)])[
    is.na(age),
    .(colnms = colnms, age = quantile(age2)),
    by = gradyear] %>%
tidyr::spread(colnms, age) %>%
select(c("gradyear", colnms))


It uses the same values across gradyears and within each gradyears.

We have to instruct explicitly to differentiate across gradyear values:

In [ ]:
teens_dt[,age2 := ifelse(is.na(age), mean(age, na.rm = T), age), by = gradyear]

In [ ]:
teens_dt[is.na(age),
            .(colnms = colnms, age = quantile(age2)),
            by = gradyear] %>%
tidyr::spread(colnms, age) %>%
select(c("gradyear", colnms))


Now values are differentiated across gradyears but homogenous within each year

Now save the values in age2 into age and delete age2 column:

In [ ]:
teens_dt[,c("age", "age2") := .(age2, NULL)]

In [ ]:
teens_dt

Now let's see whether values are imputed:

In [ ]:
teens_dt %>% dplyr::select(mdcols) %>% VIM::aggr(numbers = T)

## Data normalization

First remember the column names:

In [ ]:
names(teens_dt)

We will save the columns related with 36 keywords separately:

In [ ]:
interests <- teens_dt[,basketball:drugs]
interests

In [ ]:
interests_z <- interests[,BBmisc::normalize(.SD)]

In [ ]:
interests_z %>% sapply(quantile, na.rm = T) %>% t()

In [ ]:
interests_z[,lapply(.SD, function(x) c(mean(x), sd(x)))] %>%
                    t %>%
                    round(3) %>%
                    magrittr::set_colnames(c("mean", "sd"))
                    

We see that all means are 0 and sd's are 1. However since the values are highly skewed (many 0's and few other values) the z-scores are highly extreme

In [ ]:
props <- lapply(interests, table) %>%
    lapply(prop.table) %>%
    lapply(formattable::percent) %>%
    lapply(round, 3)

props

In [ ]:
sapply(props, "[", 1) %>%
    formattable::percent() %>%
    sort(decreasing = T)

We see that in many variables, the zero values (means no mention of the words in SNS profiles), make up more than 90% of all cases

## Visualizing distances

The euclidian distances among rows can be visualized as such (for only first 1000 rows):

In [ ]:
distancex <- factoextra::get_dist(interests_z[1:100])

In [ ]:
factoextra::fviz_dist(distancex)

Cells closer to red show more proximate observations while cells closer to blue show more distant observations

## Build and train model

We train the dataset for 5 clusters:

In [ ]:
set.seed(2345)
teen_clusters <- kmeans(interests_z, centers = 5)

In [ ]:
summary(teen_clusters)

Sizes of each cluster are:

In [ ]:
teen_clusters$size

The center values of each variable for each cluster are:

In [ ]:
centers <- teen_clusters$centers %>% t %>% round(2)
centers

Let's highlight the values in each row with significalty high or low values with kableExtra:

In [ ]:
apply(centers,
      1,
      function(x)
        {
          zs <- (x - mean(x)) / sd(x);
          cell_spec(x,
                    color = ifelse(abs(zs) > 1, "white", "black"),
                            background = ifelse(zs > 1, "navy", ifelse(zs < -1, "red", "white"))
                   )
        }
    ) %>%
t %>%
magrittr::set_colnames(1:5) %>%      
knitr::kable(escape = F) %>%
kableExtra::kable_styling() %>%
as.character() %>%
IRdisplay::display_html()

We can also visualize distinctive cluster and variable matchings with a heatmap:

In [ ]:
pheatmap::pheatmap(centers, cluster_rows = F, cluster_cols = F)

For each cluster, let's select those variables for which the cluster is above some level: 

In [ ]:
namesx <- rownames(centers)

apply(centers, 1, BBmisc::normalize) %>%
    t %>%
    plyr::alply(2, function(x) namesx[x > 1])
    #%>%
    #as.data.frame %>%
    #lapply(function(x) namesx[x > 1])

- The first cluster is above the mean for cheerleading, hollister, shopping and abercrombie. This cluster can be named as "princesses" (as per Lantz)
- The second cluster is above the mean for band and marching. This cluster can be named as "musicians" (Lantz named it as "brains")
- The 3rd cluster is above the mean for many of the sport types. This cluster can be named as "athletes"
- The 4th cluster is above the mean for hair, dress, clothes, die, death, drunk, drugs. This cluster can be named as "punks" (Lantz named it as "criminals")
- The 5th cluster is not distinctive in any of the terms. These are called "basket cases" - they were users that did not post any interests and is the largest cluster of all

We can also visualize the clusters' borders across dimensions using factroextra's fviz_cluster

Note that,  when there are more than 2 dimensions, this function automatically conducts a PCA and selects the two components that explain the most of the variance:

In [ ]:
factoextra::fviz_cluster(teen_clusters, data = interests_z, labelsize = 0)

The second component on the y axis is probably related to the intensity of sport related interests

And destructive keywords like "death" or "drugs" may be captured with the first component on the x axis

Especially "athletes" and "punks" are wide apart

## Enhance data exploration with cluster information

Now we can add the cluster info back into the original dataset

In [ ]:
teens_dt[,cluster := teen_clusters$cluster]
teens_dt

Given this new data, we can start to examine how the cluster assignment relates to individual characteristics

In [ ]:
teens_dt[,c("cluster", "gender", "age", "friends")]

We can also look at the demographic characteristics of the clusters. For example mean ages across clusters:

In [ ]:
teens_dt %>% names

In [ ]:
aggcols <- c("age", "gender_F", "friends")

teens_dt[,lapply(.SD, mean) ,.SDcols = aggcols, by = cluster][order(cluster)] %>%
magrittr::set_rownames(c("princesses", "musicians", "athletes", "punks", "basket cases")) %>%
round(2)

The mean age does not vary much by cluster, which is not too surprising as these teen identities are often determined before high school. However average age of "princesses" is slightly below and average age of "musicians" are slightly above average ages of other clusters.

The percent of females is highest in princesses and athletes clusters while lowest in musicians and basket cases

The connection between a teen's number of friends and their predicte cluster is remarkable, given that we did not use the friendship data as an input to the clustering algorithm. Also interesting is the fact that the number of friends seems to be related to the stereotype of each clusters' high school popularity; the stereotypically popular groups tend to have more friends. (Highr in "princesses" and "athletes" clusters, lowest in punks and basket cases


## Improve model performance

While conducting k-means analysis, what value should be provided as "k" - the number of clusters?

### Manual simulation

First let's dig into the model output:

In [ ]:
teen_clusters %>% listviewer::jsonedit(mode = "form")

The critical values are:
- totss (total sum of squares)
- tot.withinss (total within groups sum of squares)
- betweenss (between groups sum of squares)

As the "k" goes up withinss should leak into betweenss

In [ ]:
withinss <- sapply(1:15,
       function(x) kmeans(interests_z, centers = x) %>%
       "["(c("totss", "tot.withinss", "betweenss")) %>% unlist
       ) %>%
t %>%
as.data.table

rownames(withinss) <- 1:15

In [ ]:
withinss %>% round

In [ ]:
p1 <- withinss %>%
ggplot(aes(x = withinss[,.I], y = tot.withinss)) +
geom_line() +
xlab("Number of clusters") +
ylab("Within group sum of squares")

plotly::ggplotly(p1)

We cannot detect a clear elbow point to cut the number of clusters

### Optimal k with vegan package

Vegan package also does a simulation to determine the optimal k based on Calinski measure:

In [ ]:
modelx <- vegan::cascadeKM(interests_z, 1, 10, iter = 3)

In [ ]:
modelx$results

Calinski is a measure of between-cluster to within-cluster variance.

A plot method exists for this object, however with 30K cases, the plot takes too much time so it is avoided here. A simpler plot is as such:

In [ ]:
p2 <- modelx$results %>%
t %>%
as.data.table %>%
ggplot(aes(x = 1:10, y = calinski)) +
geom_line()

plotly::ggplotly(p2)

The k with max calinski value should be selected:

In [ ]:
which.max(modelx$results[2,])

Let's run the model with that:

In [ ]:
teen_clusters2 <- kmeans(interests_z, centers = 2)

And see the center values:

In [ ]:
centers2 <- teen_clusters2$centers %>% t %>% round(2)
centers2

And emphasize values over and above average:

In [ ]:
apply(centers2,
      1,
      function(x)
        {
          zs <- (x - mean(x)) / sd(x);
          cell_spec(x,
                    color = ifelse(abs(zs) > 0.5, "white", "black"),
                            background = ifelse(zs > 0.5, "navy", ifelse(zs < -0.5, "red", "white"))
                   )
        }
    ) %>%
t %>%
magrittr::set_colnames(1:2) %>%      
knitr::kable(escape = F) %>%
kableExtra::kable_styling() %>%
as.character() %>%
IRdisplay::display_html()

Get cluster sizes:

In [ ]:
teen_clusters2$size

Here we see that, on calinski criterion alone, the clustering is done such that first cluster is the one that holds users that did not post much interests in their profiles and the second cluster is all others

This kind of a clustering do not provide any insight at all. The reason for such an outcome is the highly skewed nature of the dataset: at least 90% of users did not have any interests in many keywords. So the optimal clustering (based on distribution of variance across between/within groups) is for values 0 versus other values.

Data exploration step is important in these situations: Knowledge of the specifics of the data will lead us not fall into the pitfall of deciding upon "numbers" alone

With a too few number of clusters, we cannot have a pattern to be interpreted
With a too large number of cluster, each cluster may not yield a distinctive insight to be acted on

### Optimal k with NbClust

NbClust package provides 30 indexes for determining the optimal number of clusters in a data set and offers the best clustering scheme from different results to the user.

However, running this function on a larger set with too large a dimension (too many variables for distance calculation) consumes too much memory

In [ ]:
dim(interests_z)

This issue is also mensioned here:

https://stats.stackexchange.com/questions/270751/nbclust-with-large-data-sets-sampling

So we will select a sample of 700 observations:

In [ ]:
samp <- teens_dt[,sample(.N, 700)]
teen_nb <- NbClust::NbClust(interests_z[samp,], min.nc = 2, max.nc = 8, index = "all", method = "kmeans")

2 cluster is selected with majority rule of 30 separate indices

The model output:

In [ ]:
teen_nb

The voting of 30 criterion can also be done manually:

In [ ]:
teen_nb$Best.nc[1,] %>% table

So NbClust also falls into the sampe pitfall as vegan did: 2 clusters are not meaningful for this dataset but is an outcome of the highly skewed nature (towards 0) of the variables